In [15]:
import pandas as pd
import numpy as np
import os
from stellargraph import datasets
from IPython.display import display, HTML
from neo4j import GraphDatabase
import torch

import time
import py2neo

functions

In [16]:
# def empty_database(graph, dataset_label):
#     # Delete nodes and relationships associated with the specified dataset label
#     delete_query = f"""
#     MATCH (n:{dataset_label}_Node)
#     DETACH DELETE n
#     """
#     tx = graph.auto()
#     tx.evaluate(delete_query)
    
#     # Drop constraints associated with the specified dataset label
#     constraints = graph.run("CALL db.constraints").data()
#     for constraint in constraints:
#         # Check if the constraint description contains the dataset label
#         if f":{dataset_label}_Node" in constraint['description']:
#             graph.run(f"DROP CONSTRAINT {constraint['name']}")

#     # Drop indexes associated with the specified dataset label
#     indexes = graph.run("CALL db.indexes").data()
#     for index in indexes:
#         # Check if the index description contains the dataset label
#         if f":{dataset_label}_Node" in index['name']:
#             graph.run(f"DROP INDEX {index['name']}")

In [17]:
# def load_nodes(graph, node_list, dataset_label):
#     query = f"""
#     UNWIND $node_list as node
#     CREATE (n:{dataset_label}_Node {{
#         subject: node.subject,
#         features: node.features
#     }})
#     SET n.ID = toInteger(node.id)
#     """
#     records = node_list.to_dict("records")
#     tx = graph.auto()
#     tx.evaluate(query, parameters={"node_list": records})


In [18]:
# def load_edges(graph, edge_list, dataset_label):
#     query = f"""
#     UNWIND $edge_list as edge
#     MATCH (source:{dataset_label}_Node {{ID: toInteger(edge.source)}})
#     MATCH (target:{dataset_label}_Node {{ID: toInteger(edge.target)}})
#     MERGE (source)-[:{dataset_label}_CITES]->(target)
#     """
#     records = edge_list.to_dict("records")
#     tx = graph.auto()
#     tx.evaluate(query, parameters={"edge_list": records})


In [19]:
# def manage_constraints(graph, dataset_label, action="create"):
#     if action == "create":
#         constraint_query = f"CREATE CONSTRAINT ON (n:{dataset_label}_Node) ASSERT n.ID IS UNIQUE"
#     elif action == "drop":
#         constraints = graph.run("CALL db.constraints").data()
#         for constraint in constraints:
#             if dataset_label in constraint['description']:
#                 graph.run(f"DROP CONSTRAINT {constraint['description']}")
#     else:
#         raise ValueError("Action must be either 'create' or 'drop'")
    
#     tx = graph.auto()
#     tx.evaluate(constraint_query)


In [28]:
# default_host = os.environ.get("EIGEN_THESIS")

# # Create the Neo4j Graph database object; the arguments can be edited to specify location and authentication
# driver = py2neo.Graph('neo4j://localhost:7687', user="neo4j", password="eigen1234")

uri = "bolt://localhost:7687"  # Adjust the URI if necessary
driver = GraphDatabase.driver(uri, auth=("neo4j", "eigen1234"))  # Replace with your credentials

PUBMED

In [21]:
# Load the CiteSeer dataset
dataset_pubmed = datasets.PubMedDiabetes()
graph, node_subjects = dataset_pubmed.load()
display(HTML(dataset_pubmed.description))
data = graph.to_adjacency_matrix(weighted=True)
labels = node_subjects.values

In [22]:
# Extract nodes and edges
nodes = list(range(len(labels)))  # Assuming nodes are indexed by integers
edges_with_values = []

In [23]:
# Create edges with their corresponding values
for source, target in np.array(np.nonzero(data)).T:
    value = data[source, target]  # Get the value from the adjacency matrix
    edges_with_values.append((source, target, value))

In [24]:
# Function to create nodes
def create_nodes(tx, labels):
    for i, label in enumerate(labels):
        tx.run("CREATE (:PubMed {id: $id, label: $label})", id=i, label=label)

# Function to create edges with values
def create_edges(tx, edges_with_values):
    for source, target, value in edges_with_values:
        tx.run("""
            MATCH (a:PubMed {id: $source}), (b:PubMed {id: $target})
            CREATE (a)-[:CITES_PubMed {value: $value}]->(b)
        """, source=source, target=target, value=value)

In [ ]:
with driver.session() as session:
    session.write_transaction(create_nodes, labels)
    session.write_transaction(create_edges, edges_with_values)

driver.close()

C:\Users\eveli\AppData\Local\Temp\ipykernel_1392\3659367915.py:2: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(create_nodes, labels)
C:\Users\eveli\AppData\Local\Temp\ipykernel_1392\3659367915.py:3: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(create_edges, edges_with_values)
Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.CartesianProductWarning} {category: } {title: This query builds a cartesian product between disconnected patterns.} {description: If a part of a query contains multiple disconnected patterns, this will build a cartesian product between all those parts. This may produce a large amount of data and slow down query processing. While occasionally intended, it may often be possible to reformulate the query that avoids the use of this cross product, perhaps by adding a relationship between the different parts or by

In [ ]:
# edge_list_pubmed = pd.read_csv(
#     os.path.join(dataset_pubmed.data_directory, "Pubmed-Diabetes.DIRECTED.cites.tab"),
#     sep="\t",
#     header=None,
#     names=["id", "source", "pipe", "target"],
#     usecols=["source", "target","id"],
# )
# # Strip the "paper:" prefix from 'source' and 'target'
# edge_list_pubmed['source'] = edge_list_pubmed['source'].str.lstrip("paper:")
# edge_list_pubmed['target'] = edge_list_pubmed['target'].str.lstrip("paper:")

# # Filter rows with non-integer values
# edge_list_pubmed = edge_list_pubmed[edge_list_pubmed['source'].str.isdigit() & edge_list_pubmed['target'].str.isdigit()]

# # Convert the filtered columns to integers
# edge_list_pubmed['source'] = edge_list_pubmed['source'].astype(int)
# edge_list_pubmed['target'] = edge_list_pubmed['target'].astype(int)

# edge_list_pubmed["label"] = "cites"


In [ ]:
# display(edge_list_pubmed.head(10))

In [ ]:
# feature_names_pubmed = ["w_{}".format(ii) for ii in range(500)]
# column_names_pubmed = feature_names_pubmed + ["subject"]
# node_list_pubmed = pd.read_csv(
#     os.path.join(dataset_pubmed.data_directory, "Pubmed-Diabetes.NODE.paper.tab"),
#     sep="\t",
#     skiprows=2,
#     header=None,
#     names=column_names_pubmed,
#     low_memory=False,
# )

In [ ]:
# # Load the dataset
# dataset_pubmed = datasets.PubMedDiabetes()
# dataset_pubmed, labels = dataset_pubmed.load()

# # Convert node features to a DataFrame
# node_list_pubmed = pd.DataFrame(dataset_pubmed.node_features(), index=dataset_pubmed.nodes())

# # Combine all feature columns into a single list for each row
# node_list_pubmed["features"] = node_list_pubmed.apply(lambda row: row.tolist(), axis=1)

# # Add labels as a separate column
# node_list_pubmed["label"] = labels.values

# # Add the node IDs as a separate column
# node_list_pubmed["id"] = dataset_pubmed.nodes()

# # Drop the individual feature columns to retain only 'features' and 'label'
# node_list_pubmed = node_list_pubmed[["id","features", "label"]]

# # Display the updated DataFrame
# node_list_pubmed.head(10)


In [ ]:
# load_nodes(graph,node_list_pubmed,'pubmed')

In [ ]:
# load_edges(graph,edge_list_pubmed,'pubmed')